In [1]:
import os 
import re
import yaml
import json
import io
import json
import gc
import yaml
import tempfile
import requests
import soundfile as sf
from multiprocessing import Pool
from huggingface_hub import hf_hub_download
from datasets import Dataset, Features, Value, load_dataset
from huggingface_hub import hf_hub_download, get_token
from tqdm import tqdm
from datasets import load_from_disk

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!uv pip install soundfile

Audited 1 package in 0.84ms


In [3]:
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 99.6 MB/s eta 0:00:00


In [2]:
from huggingface_hub import get_token

token = get_token() 

In [3]:
raw_langs = "- en   - zh   - de   - es   - ru   - ko   - fr   - ja   - pt   - tr   - pl   - ca   - nl   - ar   - sv   - it   - id   - hi   - fi   - vi   - he   - uk   - el   - ms   - cs   - ro   - da   - hu   - ta   - 'no'   - th   - ur   - hr   - bg   - lt   - la   - mi   - ml   - cy   - sk   - te   - fa   - lv   - bn   - sr   - az   - sl   - kn   - et   - mk   - br   - eu   - is   - hy   - ne   - mn   - bs   - kk   - sq   - sw   - gl   - mr   - pa   - si   - km   - sn   - yo   - so   - af   - oc   - ka   - be   - tg   - sd   - gu   - am   - yi   - lo   - uz   - fo   - ht   - ps   - tk   - nn   - mt   - sa   - lb   - my   - bo   - tl   - mg   - as   - tt   - haw   - ln   - ha   - ba   - jw   - su"

list2 = re.findall(r"[a-z]+(?:-[a-zA-Z]+)*", raw_langs)
whisper_supported = [lang.replace("'", "") for lang in list2]

In [4]:
def normalize(code):
    return code.split("-")[0]


In [6]:
data_file_path = hf_hub_download(
    repo_id="sarulab-speech/commonvoice22_sidon",
    repo_type="dataset",
    filename="paths.yaml"
)


paths = yaml.load(open(data_file_path, "r"), Loader=yaml.FullLoader)

In [7]:
valid_languages = []

for lang in paths.keys():
    base_lang = normalize(lang)
    if base_lang in whisper_supported and "test" in paths[lang]:
        valid_languages.append(lang)
print("Languages compatible", valid_languages)
print("Total:", len(valid_languages))

valid_lang_file = "valid_languages.json"
with open(valid_lang_file, "w") as f:
    json.dump(valid_languages, f, indent=2)




Languages compatible ['af', 'am', 'ar', 'as', 'az', 'ba', 'be', 'bg', 'bn', 'br', 'ca', 'cs', 'cy', 'da', 'de', 'el', 'en', 'es', 'et', 'eu', 'fa', 'fi', 'fr', 'gl', 'ha', 'he', 'hi', 'ht', 'hu', 'hy-AM', 'id', 'is', 'it', 'ja', 'ka', 'kk', 'ko', 'lo', 'lt', 'lv', 'mk', 'ml', 'mn', 'mr', 'mt', 'ne-NP', 'nl', 'nn-NO', 'oc', 'pa-IN', 'pl', 'ps', 'pt', 'ro', 'ru', 'sd', 'sk', 'sl', 'sq', 'sr', 'sv-SE', 'sw', 'ta', 'te', 'tg', 'th', 'tk', 'tr', 'tt', 'uk', 'ur', 'uz', 'vi', 'yi', 'yo', 'zh-CN', 'zh-HK', 'zh-TW']
Total: 78


In [8]:
import gc
import io
import json
import os
import tempfile
import shutil

import requests
import soundfile as sf
import yaml
from datasets import Dataset, Features, Value
from huggingface_hub import hf_hub_download
from multiprocessing import Pool
from tqdm import tqdm
from huggingface_hub import get_token

HF_TOKEN = get_token()
MAX_ROWS = 500
SPLIT = "test"
SAVE_DIR = "vc_dataset"
NUM_WORKERS = 4

with open("valid_languages.json") as f:
    valid_languages = json.load(f)

paths_file = hf_hub_download(
    repo_id="sarulab-speech/commonvoice22_sidon",
    repo_type="dataset",
    filename="paths.yaml",
    token=HF_TOKEN,
)
paths = yaml.safe_load(open(paths_file))

base_url = "https://huggingface.co/datasets/sarulab-speech/commonvoice22_sidon/resolve/main/"

os.makedirs(SAVE_DIR, exist_ok=True)

metadata_features = Features({
    "flac": Value("binary"),
    "metadata.json": {
        "accent": Value("string"),
        "age": Value("string"),
        "client_id": Value("string"),
        "down_votes": Value("int64"),
        "gender": Value("string"),
        "id": Value("string"),
        "locale": Value("string"),
        "path": Value("string"),
        "segment": Value("string"),
        "sentence": Value("string"),
        "up_votes": Value("int64"),
        "variant": Value("string"),
    },
    "__key__": Value("string"),
    "__url__": Value("string"),
})


def download_shard(url, dest_path):
    headers = {"Authorization": f"Bearer {HF_TOKEN}"}
    with requests.get(url, headers=headers, stream=True) as r:
        r.raise_for_status()
        with open(dest_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8 * 1024 * 1024):
                f.write(chunk)


def process_lang(lang):
    from datasets import load_dataset  # import inside worker to avoid multiprocessing issues

    save_path = os.path.join(SAVE_DIR, lang)
    if os.path.exists(save_path):
        print(f"{lang}: already saved, skipping")
        return

    if lang not in paths or SPLIT not in paths[lang]:
        print(f"{lang}: no test split, skipping")
        return

    shard_paths = paths[lang][SPLIT]

    all_rows = []

    for shard_rel_path in shard_paths:
        url = base_url + shard_rel_path
        with tempfile.NamedTemporaryFile(suffix=".tar.gz", delete=False) as tmp:
            tmp_path = tmp.name

        try:
            download_shard(url, tmp_path)

            ds = load_dataset(
                "webdataset",
                data_files=[tmp_path],
                streaming=True,
                features=metadata_features,
            )["train"]

            for sample in tqdm(ds, desc=f"{lang} pass1 {os.path.basename(shard_rel_path)}", leave=False):
                meta = sample.get("metadata.json", {})
                sentence = meta.get("sentence", "")
                key = sample.get("__key__")

                if not sentence:
                    continue

                all_rows.append({
                    "key": key,
                    "language": lang,
                    "source_text": sentence,
                    "upvotes": int(meta.get("up_votes", 0)),
                    "speaker_id": meta.get("client_id", ""),
                })

        except Exception as e:
            print(f"{lang}: pass1 shard {shard_rel_path} failed - {e}")
        finally:
            os.remove(tmp_path)

    if not all_rows:
        print(f"{lang}: no rows found")
        return

    all_rows.sort(key=lambda x: x["upvotes"], reverse=True)
    top_rows = all_rows[:MAX_ROWS]
    top_keys = set(r["key"] for r in top_rows)

    del all_rows
    gc.collect()

    # PASS 2: download audio and save as flac files
    audio_dir = os.path.join(save_path, "audio")
    os.makedirs(audio_dir, exist_ok=True)

    audio_map = {}

    for shard_rel_path in shard_paths:
        if len(audio_map) == len(top_keys):
            break

        url = base_url + shard_rel_path
        with tempfile.NamedTemporaryFile(suffix=".tar.gz", delete=False) as tmp:
            tmp_path = tmp.name

        try:
            download_shard(url, tmp_path)

            ds = load_dataset(
                "webdataset",
                data_files=[tmp_path],
                streaming=True,
                features=metadata_features,
            )["train"]

            for sample in tqdm(ds, desc=f"{lang} pass2 {os.path.basename(shard_rel_path)}", leave=False):
                key = sample.get("__key__")
                if key in top_keys:
                    flac_bytes = sample.get("flac")
                    try:
                        array, sampling_rate = sf.read(io.BytesIO(flac_bytes))
                        audio_filename = f"{key}.flac"
                        audio_path = os.path.join(audio_dir, audio_filename)
                        sf.write(audio_path, array, sampling_rate)
                        audio_map[key] = audio_filename
                    except Exception as e:
                        print(f"{lang}: audio decode failed for {key} - {e}")

        except Exception as e:
            print(f"{lang}: pass2 shard {shard_rel_path} failed - {e}")
        finally:
            os.remove(tmp_path)

    # assign target_text from a different row
    texts = [r["source_text"] for r in top_rows]
    for i, row in enumerate(top_rows):
        for offset in range(1, len(texts)):
            candidate = texts[(i + offset) % len(texts)]
            if candidate != row["source_text"]:
                row["target_text"] = candidate
                break
        else:
            row["target_text"] = texts[(i + 1) % len(texts)]

    final_rows = [{
        "language": r["language"],
        "source_text": r["source_text"],
        "target_text": r["target_text"],
        "upvotes": r["upvotes"],
        "speaker_id": r["speaker_id"],
        "audio_filename": audio_map.get(r["key"], ""),
    } for r in top_rows]

    dataset = Dataset.from_list(final_rows)
    dataset.save_to_disk(save_path)
    print(f"{lang}: {len(final_rows)} rows saved")

    del top_rows, final_rows, dataset, audio_map
    gc.collect()


with Pool(NUM_WORKERS) as pool:
    list(tqdm(
        pool.imap(process_lang, valid_languages),
        total=len(valid_languages),
        desc="Overall",
    ))

# Zip all audio files for easy upload
print("Zipping audio files...")
shutil.make_archive("vc_audio", "zip", SAVE_DIR)
print("Done. Upload vc_audio.zip to your storage.")

Saving the dataset (1/1 shards): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 131/131 [00:00<00:00, 36591.22 examples/s]


af: 131 rows saved


Saving the dataset (1/1 shards): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 252/252 [00:00<00:00, 71214.43 examples/s]


am: 252 rows saved


Saving the dataset (1/1 shards): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 379/379 [00:00<00:00, 107032.13 examples/s]


...

zh-HK: 500 rows saved


Saving the dataset (1/1 shards): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 165612.57 examples/s]


zh-CN: 500 rows saved


Overall: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [13:41<00:00, 10.53s/it]


Zipping audio files...
Done. Upload vc_audio.zip to your storage.


In [ ]:
#Push!!

In [ ]:
from datasets import load_from_disk
from huggingface_hub import HfApi
import os

REPO_ID = "Scicom-intl/Evaluation_Multilingual_VC"
SAVE_DIR = "vc_dataset2"
HF_TOKEN = ""

api = HfApi()

api.create_repo(repo_id=REPO_ID, repo_type="dataset", token=HF_TOKEN, exist_ok=True)


for lang in os.listdir(SAVE_DIR):
    lang_path = os.path.join(SAVE_DIR, lang)
    if not os.path.isdir(lang_path):
        continue
    try:
        ds = load_from_disk(lang_path)
        ds.push_to_hub(
            REPO_ID,
            config_name=lang,
            token=HF_TOKEN,
            private=True,
        )
        print(f"Pushed {lang} — {len(ds)} rows")
    except Exception as e:
        print(f" Failed {lang}: {e}")

api.upload_file(
    path_or_fileobj="vc_audio.zip",
    path_in_repo="vc_audio.zip",
    repo_id=REPO_ID,
    repo_type="dataset",
    token=HF_TOKEN,
)
print(" Audio zip uploaded")

In [1]:
#check whisper language support

In [ ]:
import torch
import soundfile as sf
import os
import json
from datasets import load_from_disk
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from jiwer import cer

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True
).to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

supported = []
unsupported = []
results = {}

for lang in os.listdir("vc_dataset2"):
    lang_path = os.path.join("vc_dataset2", lang)
    if not os.path.isdir(lang_path):
        continue

    try:
        ds = load_from_disk(lang_path)
        
        # Take first 5 rows
        sample_rows = [ds[i] for i in range(min(5, len(ds)))]
        
        cer_scores = []
        row_results = []

        for row in sample_rows:
            audio_path = os.path.join(lang_path, "audio", row["audio_filename"])
            result = pipe(audio_path)
            transcription = result["text"].strip()
            source = row["source_text"]
            score = cer(source, transcription)
            cer_scores.append(score)
            row_results.append({
                "expected": source,
                "got": transcription,
                "cer": round(score, 4),
            })

        avg_cer = sum(cer_scores) / len(cer_scores)
        supported_flag = avg_cer < 0.8

        print(f"\n{'wowww' if supported_flag else 'baddd'} {lang} avg CER: {avg_cer:.2f}")
        for r in row_results:
            print(f"   Expected : {r['expected']}")
            print(f"   Got      : {r['got']}")
            print(f"   CER      : {r['cer']}")

        if supported_flag:
            supported.append(lang)
        else:
            unsupported.append(lang)

        results[lang] = {
            "supported": supported_flag,
            "avg_cer": round(avg_cer, 4),
            "samples": row_results,
        }

    except Exception as e:
        print(f" {lang}: {e}")
        unsupported.append(lang)
        results[lang] = {
            "supported": False,
            "error": str(e),
        }

# Save to JSON
with open("whisper_language_support.json", "w", encoding="utf-8") as f:
    json.dump({
        "supported": supported,
        "unsupported": unsupported,
        "details": results,
    }, f, ensure_ascii=False, indent=2)

print(f"\nSupported: {len(supported)} — {supported}")
print(f"\nUnsupported: {len(unsupported)} — {unsupported}")
print("\nSaved to whisper_language_support.json")